Исправление грамматических ошибок (language_tool_python)

In [ ]:
import language_tool_python
tool = language_tool_python.LanguageTool('ru-RU')
print(tool.check("Привет как дела"))  # Должен работать без ошибок

не работает    ИСПРАВЛЕНИЕ ОШИБОК (СЛАБЫЕ СЫРЫЕ ИИ) (1 орфография spellchecker    2 пунктуация (убрал deepmultilingualpunctuation  и silero похоже не работает)  3 вся грамматика language_tool_python)     (+ pymorphy3  wordfreq)     (+ transliterate)

In [ ]:
import torch
from spellchecker import SpellChecker
import pymorphy3
import language_tool_python
from wordfreq import top_n_list

# ----------------------
# 1. Орфография (pymorphy3 + spellchecker + wordfreq)
# ----------------------
morph = pymorphy3.MorphAnalyzer()

# Загружаем частотный список русских слов (например, топ-100000)
ru_words = top_n_list("ru", 100000)

spell = SpellChecker(language=None)
spell.word_frequency.load_words(ru_words)


def correct_spelling(text: str) -> str:
    words = text.split()
    corrected = []
    for w in words:
        lw = w.lower()

        # Проверяем слово через pymorphy3
        parses = morph.parse(lw)
        if parses and parses[0].score > 0.5:
            corrected.append(w)
            continue

        # Если неизвестно — пробуем spellchecker
        cand = spell.correction(lw)
        if cand is None:
            corrected.append(w)
        else:
            if w[0].isupper():
                cand = cand.capitalize()
            corrected.append(cand)

    return " ".join(corrected)


# ----------------------
# 2. Пунктуация (Silero)
# ----------------------
model, example_texts, languages, punct, apply_te = torch.hub.load(
    repo_or_dir="snakers4/silero-models",
    model="silero_te"
)


def restore_punctuation(text: str) -> str:
    return apply_te(text, lan="ru")


# ----------------------
# 3. Грамматика (LanguageTool)
# ----------------------
tool = language_tool_python.LanguageTool("ru-RU")


def correct_grammar(text: str) -> str:
    matches = tool.check(text)
    return language_tool_python.utils.correct(text, matches)


# ----------------------
# 4. Конвейер
# ----------------------
def process_text_strict(text: str) -> str:
    text = restore_punctuation(text)
    text = correct_spelling(text)
    text = correct_grammar(text)
    return text


# ----------------------
# Пример
# ----------------------
raw_text = "привет как дела я сегодн пошол в магазин а патом паехал дамой"
processed = process_text_strict(raw_text)
print(processed)